In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-04-22 15:43:45.556756: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-22 15:43:45.638468: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-22 15:43:46.550112: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
# Directory paths for training, validation, and test data
train_dir = 'train'  # path to the directory containing training images

In [12]:
datagen = ImageDataGenerator(
    rescale=1./255,  # rescale pixel values to [0, 1]
    horizontal_flip=True  # augment with horizontal flip
)

In [13]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # resize images to (224, 224) as required by ResNet-50
    batch_size=32,
    class_mode='categorical'  # set to 'categorical' for multi-class classification
)

Found 6 images belonging to 3 classes.


In [15]:
train_generator.class_indices

{'.ipynb_checkpoints': 0, 'pizza': 1, 'shawarma': 2}

In [21]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

In [22]:
# Load the pretrained ResNet-50 model without the top (fully connected) layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model to prevent its weights from being updated during training
base_model.trainable = False


In [23]:
class_labels = decode_predictions(np.eye(1, 1000, 1), top=1000)[0]
class_labels = [label[1] for label in class_labels]
print("ImageNet classes: ", class_labels)

ImageNet classes:  ['goldfish', 'toilet_tissue', 'sea_urchin', 'hog', 'zebra', 'sorrel', 'guinea_pig', 'beaver', 'marmot', 'fox_squirrel', 'porcupine', 'hamster', 'Angora', 'hare', 'wood_rabbit', 'sea_cucumber', 'starfish', 'warthog', 'lycaenid', 'sulphur_butterfly', 'cabbage_butterfly', 'monarch', 'ringlet', 'admiral', 'damselfly', 'dragonfly', 'lacewing', 'leafhopper', 'cicada', 'mantis', 'cockroach', 'wild_boar', 'hippopotamus', 'cricket', 'ox', 'baboon', 'patas', 'guenon', 'siamang', 'gibbon', 'chimpanzee', 'gorilla', 'orangutan', 'three-toed_sloth', 'armadillo', 'badger', 'skunk', 'otter', 'black-footed_ferret', 'polecat', 'mink', 'weasel', 'llama', 'Arabian_camel', 'gazelle', 'impala', 'hartebeest', 'ibex', 'bighorn', 'ram', 'bison', 'water_buffalo', 'walking_stick', 'grasshopper', 'langur', 'grey_fox', 'kit_fox', 'red_fox', 'hyena', 'African_hunting_dog', 'dhole', 'dingo', 'coyote', 'red_wolf', 'white_wolf', 'timber_wolf', 'Mexican_hairless', 'standard_poodle', 'miniature_poodle

In [24]:
# Number of classes in your dataset, including pizza, shawarma, and ImageNet classes
num_classes = 3
# Add custom output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [25]:
# Create final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [27]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
model.fit_generator(
train_generator,
steps_per_epoch=len(train_generator),
epochs=10,
validation_data=None) #

/tmp/ipykernel_39734/966224173.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(
2023-04-22 16:04:47.352272: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 2.2082 - accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 373ms/step - loss: 0.8264 - accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 352ms/step - loss: 0.6269 - accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 335ms/step - loss: 0.5784 - accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 384ms/step - loss: 0.5362 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 389ms/step - loss: 0.5029 - accuracy: 0.6667
Epoch 7/10
1/1 [==============================] - 0s 382ms/step - loss: 0.4795 - accuracy: 0.6667
Epoch 8/10
1/1 [==============================] - 0s 386ms/step - loss: 0.4540 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 376ms/step - loss: 0.4359 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 365ms/step - loss: 0.4122 - accuracy: 1.0000


In [42]:


# Load a test image
image_path = 'train/shawarma/shawarma_1.png'
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input


image = load_img(image_path, target_size=(224, 224))

# Convert the image to a numpy array
image_array = img_to_array(image)

# Preprocess the image
image_array = preprocess_input(image_array)
image_array = image_array.reshape((1,) + image_array.shape)

# Make prediction
prediction = model.predict(image_array)

# Get the class label with highest probability
class_index = prediction.argmax()
# class_label = class_names[class_index]  # assuming class_names is a list of class labels

# Get the probability of the predicted class
probability = prediction[0, 1]

# Print the predicted class label and probability
print(f'Predicted Class:  Probability: {probability:.2f}')



1/1 [==============================] - 0s 106ms/step
Predicted Class:  Probability: 0.99
